In [12]:
import pandas as pd
import re

In [3]:
org_ntw = pd.read_csv('../Nov.25__smartlog.scrapper/smlog_data/네트워크_merged.csv')
org_md = pd.read_csv('../Nov.25__smartlog.scrapper/smlog_data/미디어_merged.csv')
org_site = pd.read_csv('../Nov.25__smartlog.scrapper/smlog_data/사이트_merged.csv')
org_kwd = pd.read_csv('../Nov.25__smartlog.scrapper/smlog_data/키워드_merged.csv')

In [15]:
ntw_columns = ['media','network','totalClick','validClick','churnRate','avgSesstionTime','conv_ask','conv_join','conv_order','conv_rev','date']
kwd_columns = ['media','keyword','totalClick','validClick','churnRate','avgSesstionTime','conv_ask','conv_join','conv_order','conv_rev','date']
site_columns = ['media','site','totalClick','validClick','churnRate','avgSesstionTime','conv_ask','conv_join','conv_order','conv_rev','date']
md_columns = ['media','totalClick','validClick','churnRate','avgSesstionTime','conv_ask','conv_join','conv_order','conv_rev','date']

org_ntw.columns = ntw_columns
org_kwd.columns = kwd_columns
org_site.columns = site_columns
org_md.columns = md_columns


# 예시: churnRate, conv_rev에서 %와 ₩를 제거하고 int로 변환하도록 apply 함수 사용
for df in [org_ntw, org_kwd, org_site, org_md]:
    if 'churnRate' in df.columns:
        df['churnRate'] = df['churnRate'].apply(lambda x: int(str(x).replace('%', '')))
    if 'conv_rev' in df.columns:
        df['conv_rev'] = df['conv_rev'].apply(lambda x: int(str(x).replace('₩', '').replace(',', '')))
    df['date'] = pd.to_datetime(df['date'])

org_ntw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   media            162 non-null    object        
 1   network          162 non-null    object        
 2   totalClick       162 non-null    int64         
 3   validClick       162 non-null    int64         
 4   churnRate        162 non-null    int64         
 5   avgSesstionTime  162 non-null    object        
 6   conv_ask         162 non-null    int64         
 7   conv_join        162 non-null    int64         
 8   conv_order       162 non-null    int64         
 9   conv_rev         162 non-null    int64         
 10  date             162 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(7), object(3)
memory usage: 14.1+ KB


,media,network,totalClick,validClick,churnRate,avgSesstionTime,conv_ask,conv_join,conv_order,conv_rev,date
0,구글,Google 검색,3641,2838,97%,0:00:49,71,186,458,₩275,2025-11-05
1,네이버,파워링크,1204,1026,81%,0:03:01,40,21,50,₩9,2025-11-05
2,구글,Google 검색,3641,2838,97%,0:00:49,71,186,458,₩275,2025-11-06
3,네이버,파워링크,1204,1026,81%,0:03:01,40,21,50,₩9,2025-11-06
4,구글,Google 검색,3312,2564,97%,0:00:49,65,161,412,₩248,2025-08-18
...,...,...,...,...,...,...,...,...,...,...,...
157,네이버,파워링크,1124,959,80%,0:03:03,36,19,45,₩9,2025-11-02
158,구글,Google 검색,3312,2564,97%,0:00:49,65,161,412,₩248,2025-11-03
159,네이버,파워링크,1124,959,80%,0:03:03,36,19,45,₩9,2025-11-03
160,구글,Google 검색,3312,2564,97%,0:00:49,65,161,412,₩248,2025-11-04


In [27]:
# org_ntw.groupby(['media','network']).agg({'totalClick': 'sum', 'validClick': 'sum', 'conv_ask': 'sum', 'conv_join': 'sum', 'conv_order': 'sum', 'conv_rev': 'sum'}).sort_values(by='conv_rev', ascending=False)
org_site.groupby(['media']).agg({'totalClick': 'sum', 'validClick': 'sum', 'conv_ask': 'sum', 'conv_join': 'sum', 'conv_order': 'sum', 'conv_rev': 'sum'}).sort_values(by='conv_rev', ascending=False)
org_md.groupby(['media']).agg({'totalClick': 'sum', 'validClick': 'sum', 'conv_ask': 'sum', 'conv_join': 'sum', 'conv_order': 'sum', 'conv_rev': 'sum'}).sort_values(by='conv_rev', ascending=False)
org_kwd.groupby(['media','keyword']).agg({'totalClick': 'sum', 'validClick': 'sum', 'conv_ask': 'sum', 'conv_join': 'sum', 'conv_order': 'sum', 'conv_rev': 'sum'}).sort_values(by='conv_rev', ascending=False)

totalClick  validClick  conv_ask  conv_join  \
media keyword                                                                
네이버   여의도센텀치과                         405         324         0         81   
      측두근보톡스                          162         162         0         81   
      턱관절                            2432        2351        81        243   
      서울센텀턱구강내과치과의원                   405         324        81          0   
      턱관절통증치료                        1950        1950         0          0   
      턱관절치료병원                        8193        6654        81        324   
      구강내과전문의                        6399        5508       324          0   
      여의도이갈이치료                       1134         891        81          0   
      이갈이원인                           162         162         0          0   
      이수역턱관절                            2           2         0          0   
      이갈이치료                           893         731         0          0   
      이갈이치과                            81          81         0          0   
      SEOULDENTALCLINICGIMHAE          81          81         0          0   
      이갈이스플린트                         162         162         0          0   
      인천턱관절                           162         162         0          0   
      이갈이보톡스후기                         81           0         0          0   
      이갈이보톡스실비                         81          81         0          0   
      이갈이보톡스                          243         162         0          0   
      이갈이방지마우스피스                      245         245         0          0   
      인천구강내과                          245         164         0          0   
      인천턱빠짐                            81          81         0          0   
      인천턱관절병원                         407         245         0          0   
      인천턱관절치료병원                       648         567        81          0   
      이갈이마우스피스                       1215        1215         0          0   
      일산구강내과                          731         488        81          0   
      일산턱관절                           162         162         0          0   
      잠실턱관절                            81          81         0          0   
      주변턱관절전문병원                        81          81         0          0   
      치과구강내과서울                         81          81         0          0   
      치과마우스피스                         243         243         0          0   
      치과보톡스                           243         243         0          0   
      치의학과                             81          81         0          0   
      코골이                             324         243         0          0   
      코골이치료                            81          81         0          0   
      이갈이방지                            81          81         0          0   
      의료진소개                           243         243         0          0   
      이갈이고치는법                         162         162         0          0   
      연세구강턱내과치과의원                      81          81         0          0   
      양천구구강내과                          81          81         0          0   
      양천구턱관절치료                         81          81         0          0   
      얼굴비대칭교정                          81          81         0          0   
      여의도구강내과                         162         162         0          0   
      여의도구강내과치과                        81          81         0          0   
      여의도센텀구강                         162         162         0          0   
      여의도센텀구강내과치과                      81          81         0          0   
      여의도이갈이                          328         328         0          0   
      여의도턱관절                           81          81         0          0   
      여의도턱관절병원                         81          81        81          0   
      여의도턱관절치과                        243         243         0          0   
      여의도턱통증병원                        162         162

In [26]:
pd.set_option('display.max_rows', None)

In [20]:
org_site.media.unique()

array(['네이버', '구글'], dtype=object)